In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Load and preprocess dataset
df = pd.read_csv('diabetes.csv')  # Replace with your actual file path

# Fill missing values if needed
df['Pregnancies'] = df['Pregnancies'].fillna(0)
df['BloodPressure'] = df['BloodPressure'].fillna(0)
df['Glucose'] = df['Glucose'].fillna(0)

# Prepare features and labels
X = df[['Pregnancies', 'BloodPressure', 'Glucose']].values
y = df['Outcome'].values  # Assuming 'Outcome' is the target column

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model
input_dim = X_train.shape[1]
output_dim = len(set(y))  # Number of unique classes
model = SimpleNN(input_dim, output_dim)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    val_outputs = model(X_val_tensor)
    _, predicted = torch.max(val_outputs, 1)
    print(classification_report(y_val_tensor.numpy(), predicted.numpy()))


Epoch [1/10], Loss: 0.7555
Epoch [2/10], Loss: 0.7443
Epoch [3/10], Loss: 0.7339
Epoch [4/10], Loss: 0.7243
Epoch [5/10], Loss: 0.7155
Epoch [6/10], Loss: 0.7072
Epoch [7/10], Loss: 0.6994
Epoch [8/10], Loss: 0.6922
Epoch [9/10], Loss: 0.6854
Epoch [10/10], Loss: 0.6790
              precision    recall  f1-score   support

           0       0.83      0.48      0.61        99
           1       0.47      0.82      0.60        55

    accuracy                           0.60       154
   macro avg       0.65      0.65      0.60       154
weighted avg       0.70      0.60      0.61       154

